<a href="https://colab.research.google.com/github/Yukselendincer/datasceinceproject/blob/main/Makine_%C3%96%C4%9Frenimi_%2B_Ajan_Tabanl%C4%B1_Yapay_Zeka_(Lojistik).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Hücre: Kurulum ve Kütüphaneler

In [ ]:
# Gerekli Kütüphanelerin Yüklenmesi
!pip install xgboost lightgbm scikit-learn pandas numpy matplotlib seaborn

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import json
import warnings
from datetime import datetime, timedelta
from collections import defaultdict, deque

# ML Kütüphaneleri
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, mean_squared_error, r2_score, f1_score, roc_auc_score
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingRegressor, IsolationForest
from sklearn.cluster import KMeans
import xgboost as xgb
import lightgbm as lgb

warnings.filterwarnings('ignore')
print("Kütüphaneler yüklendi.")

2. Hücre: Sentetik Veri Üretimi ve Hazırlık

In [ ]:
# --- VERİ SETİ SİMÜLASYONU (SQL Yerine) ---
def generate_synthetic_data(n=2000):
    np.random.seed(42)
    data = {
        'shipment_id': [f"MAEU{i:08d}" for i in range(n)],
        'booking_date': pd.date_range(start='2023-01-01', periods=n, freq='H'),
        'origin_region': np.random.choice(['Asia', 'Europe', 'North_America', 'South_Asia'], n),
        'dest_region': np.random.choice(['Asia', 'Europe', 'North_America', 'Middle_East'], n),
        'gross_weight_kg': np.random.uniform(1000, 30000, n),
        'volume_cbm': np.random.uniform(5, 75, n),
        'temperature_controlled': np.random.choice([0, 1], n, p=[0.8, 0.2]),
        'hazardous_cargo': np.random.choice([0, 1], n, p=[0.95, 0.05]),
        'fragile_cargo': np.random.choice([0, 1], n, p=[0.9, 0.1]),
        'container_category': np.random.choice(['DRY', 'REEFER', 'SPECIAL'], n),
        'container_code': np.random.choice(['20DRY', '40HC', '40HCRF', '20RF'], n),
        'carrier': np.random.choice(['Maersk', 'MSC', 'CMA-CGM', 'Hapag-Lloyd'], n),
        'route_distance_nm': np.random.uniform(3000, 12000, n),
        'commodity_category': np.random.choice(['electronics', 'fresh_produce', 'machinery', 'textiles'], n),
        'packaging_type': np.random.choice(['carton', 'pallet', 'crate'], n),
        'num_pallets': np.random.randint(0, 20, n),
        'num_packages': np.random.randint(10, 500, n),
        'value_tier': np.random.choice([1, 2, 3], n),
        'transit_days_actual': np.random.uniform(10, 45, n),
        'delayed_flag': np.random.choice([0, 1], n, p=[0.8, 0.2]),
        'booking_lead_days': np.random.randint(1, 30, n),
        'is_damaged': np.random.choice([0, 1], n, p=[0.95, 0.05]), # Hasar hedefi
        'customs_hours': np.random.uniform(2, 72, n)
    }

    # Türetilmiş sütunlar
    df = pd.DataFrame(data)
    df['booking_month'] = df['booking_date'].dt.month
    df['is_peak_season'] = df['booking_month'].isin([9, 10, 11, 12]).astype(int)
    df['booking_weekday'] = df['booking_date'].dt.dayofweek

    # Konteyner özelliklerini simüle et
    df['container_count'] = np.random.randint(1, 3, n)
    df['total_teu'] = df['container_count'] * 2 # Basit varsayım
    df['container_fill_pct'] = np.random.uniform(50, 100, n)
    df['density_kg_per_cbm'] = df['gross_weight_kg'] / df['volume_cbm']
    df['container_max_payload_kg'] = 28000
    df['container_volume_cbm'] = 67.0

    # Navlun Maliyeti (Target)
    df['freight_cost_usd'] = (
        1500 * df['total_teu'] +
        df['route_distance_nm'] * 0.15 +
        df['temperature_controlled'] * 800 +
        np.random.normal(0, 200, n)
    ).clip(lower=500)

    # Aşırı ağırlık hedefi
    df['is_overweight'] = (df['gross_weight_kg'] > df['container_max_payload_kg']).astype(int)

    # Konsolidasyon potansiyeli
    df['consolidation_potential'] = ((df['container_fill_pct'] < 60) & (df['container_count'] == 1)).astype(int)

    # Karbon emisyonu
    df['carbon_emission_kg'] = df['route_distance_nm'] * 0.025 * df['gross_weight_kg'] / 1000

    # Müşteri memnuniyeti
    df['satisfaction_score'] = np.random.uniform(1, 5, n)

    return df

# Veriyi yükle
df = generate_synthetic_data(2000)
print(f"Veri seti oluşturuldu: {df.shape}")

3. Hücre: ML Modellerinin Eğitimi

In [ ]:
# --- MODEL EĞİTİMİ ---

# 1. Navlun Maliyeti Tahmini (Regression)
print("Model 1: Maliyet Tahmini eğitiliyor...")
features_cost = ['route_distance_nm', 'gross_weight_kg', 'volume_cbm',
                 'temperature_controlled', 'hazardous_cargo', 'container_count']
X_cost = df[features_cost]
y_cost = df['freight_cost_usd']

prep_cost = ColumnTransformer([
    ('num', StandardScaler(), features_cost)
])

model_cost = Pipeline([
    ('prep', prep_cost),
    ('reg', xgb.XGBRegressor(n_estimators=50, random_state=42))
])
model_cost.fit(X_cost, y_cost)

# 2. ETA Tahmini (Regression)
print("Model 2: ETA Tahmini eğitiliyor...")
features_eta = ['route_distance_nm', 'hazardous_cargo', 'temperature_controlled']
cat_features_eta = ['origin_region', 'dest_region', 'carrier']

X_eta = df[features_eta + cat_features_eta]
y_eta = df['transit_days_actual']

prep_eta = ColumnTransformer([
    ('num', StandardScaler(), features_eta),
    ('cat', OneHotEncoder(handle_unknown='ignore'), cat_features_eta)
])

model_eta = Pipeline([
    ('prep', prep_eta),
    ('reg', GradientBoostingRegressor(n_estimators=50, random_state=42))
])
model_eta.fit(X_eta, y_eta)

# 3. Gecikme Olasılığı (Classification)
print("Model 3: Gecikme Olasılığı eğitiliyor...")
# Basitleştirilmiş özellikler
features_delay = ['route_distance_nm', 'booking_month']
cat_delay = ['carrier', 'origin_region']
X_delay = df[features_delay + cat_delay]
y_delay = df['delayed_flag']

prep_delay = ColumnTransformer([
    ('num', StandardScaler(), features_delay),
    ('cat', OneHotEncoder(handle_unknown='ignore'), cat_delay)
])

model_delay = Pipeline([
    ('prep', prep_delay),
    ('clf', xgb.XGBClassifier(n_estimators=50, scale_pos_weight=3, random_state=42))
])
model_delay.fit(X_delay, y_delay)

# 4. Hasar Olasılığı (Classification)
print("Model 4: Hasar Riski eğitiliyor...")
features_dmg = ['transit_days_actual', 'container_fill_pct']
cat_dmg = ['packaging_type']
X_dmg = df[features_dmg + cat_dmg]
y_dmg = df['is_damaged']

prep_dmg = ColumnTransformer([
    ('num', StandardScaler(), features_dmg),
    ('cat', OneHotEncoder(handle_unknown='ignore'), cat_dmg)
])
model_damage = Pipeline([
    ('prep', prep_dmg),
    ('clf', RandomForestClassifier(n_estimators=50, class_weight='balanced'))
])
model_damage.fit(X_dmg, y_dmg)

# 5. Gümrük ve Karbon (Basit Regresyonlar)
# Karbon
model_carbon = Pipeline([
    ('prep', ColumnTransformer([('num', StandardScaler(), ['route_distance_nm', 'gross_weight_kg'])])),
    ('reg', lgb.LGBMRegressor(verbosity=-1))
])
model_carbon.fit(df[['route_distance_nm', 'gross_weight_kg']], df['carbon_emission_kg'])

# Gümrük
model_customs = Pipeline([
    ('prep', ColumnTransformer([('cat', OneHotEncoder(handle_unknown='ignore'), ['dest_region'])])),
    ('reg', RandomForestRegressor(n_estimators=10))
])
model_customs.fit(df[['dest_region']], df['customs_hours'])

# Konsolidasyon (Sınıflandırma)
model_consolidation = Pipeline([
    ('prep', ColumnTransformer([('num', StandardScaler(), ['container_fill_pct', 'volume_cbm'])])),
    ('clf', lgb.LGBMClassifier(verbosity=-1))
])
model_consolidation.fit(df[['container_fill_pct', 'volume_cbm']], df['consolidation_potential'])

# Müşteri Segmentasyonu (KMeans)
scaler_cluster = StandardScaler()
X_clust = df[['gross_weight_kg', 'freight_cost_usd']]
model_cluster = KMeans(n_clusters=3, random_state=42, n_init=10)
model_cluster.fit(scaler_cluster.fit_transform(X_clust))

print("Tüm modeller eğitildi.")

4. Hücre: Ajanlar, Ayrıştırıcılar ve Ana Pipeline

In [ ]:
# --- YARDIMCI FONKSİYONLAR VE AJANLAR ---

# 1. Regex Tabanlı "GenAI" Ayrıştırıcılar [cite: 3083]
CARGO_KEYWORDS = {
    'electronics': ['laptop', 'phone', 'tv', 'computer', 'electronic'],
    'fresh_produce': ['apple', 'banana', 'fruit', 'vegetable', 'produce'],
    'frozen_food': ['frozen', 'ice cream', 'meat', 'seafood', 'fish', 'tuna'],
    'machinery': ['machine', 'equipment', 'industrial', 'excavator'],
    'chemicals': ['chemical', 'acid', 'hazardous']
}

def parse_cargo_text(text):
    t = text.lower()
    cargo_type = 'general_cargo'
    for ctype, kws in CARGO_KEYWORDS.items():
        if any(kw in t for kw in kws):
            cargo_type = ctype
            break

    # Ağırlık
    weight_kg = 0
    m = re.search(r'(\d+(?:\.\d+)?)\s*(?:kg|kilo)', t)
    if m: weight_kg = float(m.group(1))
    else:
        m = re.search(r'(\d+(?:\.\d+)?)\s*ton', t)
        if m: weight_kg = float(m.group(1)) * 1000

    # Hacim
    volume_cbm = 0
    m = re.search(r'(\d+(?:\.\d+)?)\s*(?:cbm|m3)', t)
    if m: volume_cbm = float(m.group(1))

    # Sıcaklık
    temp = None
    m = re.search(r'(-?\d+)\s*°?\s*[cC]', t)
    if m: temp = float(m.group(1))

    hazardous = 1 if any(w in t for w in ['hazardous', 'dangerous']) else 0
    fragile = 1 if any(w in t for w in ['fragile', 'delicate']) else 0

    return {
        'cargo_type': cargo_type, 'weight_kg': weight_kg, 'volume_cbm': volume_cbm,
        'temperature_c': temp, 'hazardous': hazardous, 'fragile': fragile
    }

# Rota Ayrıştırıcı [cite: 3162]
def parse_route(route_text):
    # Basitleştirilmiş harita
    PORT_MAP = {
        'mumbai': ('South_Asia'), 'rotterdam': ('Europe'),
        'shanghai': ('Asia'), 'los angeles': ('North_America'),
        'singapore': ('Asia'), 'new york': ('North_America'),
        'dubai': ('Middle_East')
    }
    t = route_text.lower()
    origin_region = 'Asia' # Default
    dest_region = 'Europe' # Default
    origin_name = "Unknown"
    dest_name = "Unknown"

    found = []
    for port, region in PORT_MAP.items():
        if port in t:
            found.append((port, region))

    if len(found) >= 2:
        origin_name, origin_region = found[0]
        dest_name, dest_region = found[1]

    # Mesafe (Basit)
    dist = 5000 if origin_region != dest_region else 1000

    return {
        'origin': {'name': origin_name, 'region': origin_region},
        'destination': {'name': dest_name, 'region': dest_region},
        'distance_nm': dist
    }

# Konteyner Önerisi [cite: 3211]
def recommend_container(cargo):
    temp = cargo.get('temperature_c')
    vol = cargo.get('volume_cbm', 0)

    code = '40HC'
    name = '40ft High Cube'

    if temp is not None:
        code = '40HCRF'
        name = '40ft Reefer'
    elif vol < 30:
        code = '20DRY'
        name = '20ft Dry'

    count = max(1, int(np.ceil(vol / 65)))
    return {'code': code, 'name': name, 'count': count, 'fill_pct': 80.0}

# --- AJAN SINIFLARI [cite: 4436] ---

class ReasoningAgent:
    def think(self, cargo, route):
        steps = []
        steps.append(f"Kargo analizi: {cargo['cargo_type']}")
        if cargo['temperature_c']:
            steps.append(f"Sıcaklık kontrolü gerekli: {cargo['temperature_c']}C")
        steps.append(f"Rota: {route['origin']['name']} -> {route['destination']['name']}")
        return {'steps': steps}

class NegotiationAgent:
    def negotiate(self, cost):
        # Basit indirim mantığı
        discount = cost * 0.05
        return {'final': cost - discount, 'saved': discount}

class RiskAgent:
    def assess(self, predictions):
        risks = []
        if predictions['delay_prob'] > 0.15:
            risks.append("Yüksek gecikme riski")
        if predictions['damage_prob'] > 0.05:
            risks.append("Hasar riski")

        level = 'HIGH' if len(risks) > 0 else 'LOW'
        return {'risks': risks, 'level': level}

class DecisionAgent:
    def decide(self, risk_level):
        return 'APPROVED' if risk_level != 'HIGH' else 'REVIEW_REQUIRED'

# --- ANA PİPELINE [cite: 4530] ---

def maersk_genai_pipeline(text, route_text):
    print("="*40)
    print("MAERSK AJAN TABANLI AI SİSTEMİ BAŞLATILIYOR")
    print(f"Girdi: {text}")
    print(f"Rota: {route_text}")
    print("-" * 20)

    # 1. Ayrıştırma
    cargo = parse_cargo_text(text)
    route = parse_route(route_text)
    print(f"[1] Kargo Ayrıştırıldı: {cargo}")

    # 2. Öneri
    container = recommend_container(cargo)
    print(f"[2] Konteyner Önerisi: {container['count']}x {container['name']}")

    # 3. ML Tahminleri
    # Girdi DataFrame oluşturma
    input_df = pd.DataFrame([{
        'route_distance_nm': route['distance_nm'],
        'gross_weight_kg': cargo['weight_kg'],
        'volume_cbm': cargo['volume_cbm'],
        'temperature_controlled': 1 if cargo['temperature_c'] else 0,
        'hazardous_cargo': cargo['hazardous'],
        'container_count': container['count'],
        'booking_month': datetime.now().month,
        'origin_region': route['origin']['region'],
        'dest_region': route['destination']['region'],
        'carrier': 'Maersk',
        'transit_days_actual': 20, # Dummy for damage model input
        'container_fill_pct': container['fill_pct'],
        'packaging_type': 'pallet' # Default
    }])

    cost_pred = model_cost.predict(input_df[features_cost])[0]
    eta_pred = model_eta.predict(input_df[features_eta + cat_features_eta])[0]

    # Gecikme için
    delay_input = input_df[features_delay + cat_delay]
    delay_prob = model_delay.predict_proba(delay_input)[0][1]

    # Hasar için
    dmg_input = input_df[features_dmg + cat_dmg]
    damage_prob = model_damage.predict_proba(dmg_input)[0][1]

    predictions = {
        'cost': cost_pred, 'eta': eta_pred,
        'delay_prob': delay_prob, 'damage_prob': damage_prob
    }

    print(f"[3] ML Tahminleri: Maliyet=${cost_pred:.2f}, ETA={eta_pred:.1f} gün, Gecikme Riski=%{delay_prob*100:.1f}")

    # 4. Ajanların Çalışması
    reasoner = ReasoningAgent()
    negotiator = NegotiationAgent()
    risk_assessor = RiskAgent()
    decision_maker = DecisionAgent()

    logic = reasoner.think(cargo, route)
    price = negotiator.negotiate(cost_pred)
    risks = risk_assessor.assess(predictions)
    decision = decision_maker.decide(risks['level'])

    print("\n[4] Ajan Kararları:")
    print(f"   - Mantık: {logic['steps']}")
    print(f"   - Müzakere: Son Fiyat ${price['final']:.2f} (Kazanç: ${price['saved']:.2f})")
    print(f"   - Risk Analizi: {risks['level']} ({risks['risks']})")
    print(f"   - NİHAİ KARAR: {decision}")
    print("="*40)

    return {'decision': decision, 'final_cost': price['final']}

# --- TEST ET --- [cite: 4674]
test_text = "Ship 12 tons of fresh apples, requires 4°C, fragile, volume 25 cbm"
test_route = "Mumbai to Rotterdam"

result = maersk_genai_pipeline(test_text, test_route)

5. Hücre: Gerçek Senaryo Testleri

In [ ]:
# --- SENARYO TESTLERİ ---

def run_scenarios():
    print("\n" + "="*20 + " SENARYO TESTLERİ BAŞLIYOR " + "="*20 + "\n")

    # Senaryo 1: Taze Elma (Soğutma ve Kırılganlık)
    print(">>> SENARYO 1: TAZE GIDA")
    maersk_genai_pipeline(
        text="Ship 12 tons of fresh apples, requires 4°C, fragile, volume 25 cbm",
        route_text="Mumbai to Rotterdam"
    )

    # Senaryo 2: Donmuş Orkinos (Derin Dondurucu ve Yüksek Değer)
    print("\n>>> SENARYO 2: DONMUŞ GIDA (YÜKSEK RİSK)")
    maersk_genai_pipeline(
        text="Sashimi-grade tuna, 8000 kg, needs -50°C freezing, 15 cbm volume",
        route_text="Shanghai to Los Angeles"
    )

    # Senaryo 3: Elektronik (Yüksek Değer ve Kırılgan)
    print("\n>>> SENARYO 3: ELEKTRONİK")
    maersk_genai_pipeline(
        text="Laptop computers and phones, fragile electronics, 5000 kg, 45 cbm",
        route_text="Singapore to New York"
    )

    # Senaryo 4: Tehlikeli Madde (Ekstra Güvenlik)
    print("\n>>> SENARYO 4: TEHLİKELİ MADDE")
    maersk_genai_pipeline(
        text="Industrial chemicals, hazardous, 20000 kg, 30 cbm, dangerous goods",
        route_text="Rotterdam to Dubai"
    )

# Testleri Çalıştır
run_scenarios()

6. Hücre: Gelişmiş "Bilişsel Hafıza" Ajanı (Cognitive Memory Agent)

In [ ]:
# --- BİLİŞSEL HAFIZA AJANI (Advanced Agent) ---
from collections import deque

class CognitiveMemoryAgent:
    def __init__(self, memory_capacity=1000):
        # Epizodik hafıza: Son yaşanan olayları tutar
        self.episodic_memory = deque(maxlen=memory_capacity)
        # Semantik hafıza: Öğrenilen genel kalıpları tutar
        self.semantic_memory = {
            'successful_routes': defaultdict(int),
            'problematic_carriers': defaultdict(int)
        }

    def store_episode(self, episode):
        """Yeni bir sevkiyat deneyimini hafızaya kaydeder."""
        self.episodic_memory.append(episode)

        # Başarı/Başarısızlık durumuna göre öğrenme
        route_key = f"{episode['origin']}->{episode['dest']}"
        if episode['outcome'] == 'SUCCESS':
            self.semantic_memory['successful_routes'][route_key] += 1
        elif episode['outcome'] == 'DELAYED':
            self.semantic_memory['problematic_carriers'][episode['carrier']] += 1

    def recall_similar(self, current_route, current_cargo):
        """Benzer geçmiş deneyimleri hatırlar."""
        similar_cases = []
        for case in self.episodic_memory:
            if case['cargo_type'] == current_cargo and \
               case['origin'] == current_route['origin']['name']:
                similar_cases.append(case)
        return similar_cases

    def get_advice(self, current_route, carrier):
        """Geçmişe dayanarak tavsiye verir."""
        advice = []

        # Taşıyıcı kontrolü
        failures = self.semantic_memory['problematic_carriers'].get(carrier, 0)
        if failures > 2:
            advice.append(f"UYARI: {carrier} ile bu rotada daha önce {failures} kez gecikme yaşandı.")

        # Rota başarısı
        route_key = f"{current_route['origin']['name']}->{current_route['destination']['name']}"
        successes = self.semantic_memory['successful_routes'].get(route_key, 0)
        if successes > 5:
            advice.append(f"ONAY: Bu rota ({route_key}) oldukça güvenilir görünüyor.")

        return advice

# --- HAFIZA AJANINI TEST ETME ---

print("="*40)
print("BİLİŞSEL HAFIZA SİMÜLASYONU")
print("="*40)

# 1. Hafızayı Eğit (Geçmiş Verilerle Doldur)
memory_agent = CognitiveMemoryAgent()

# Geçmişte yaşanmış olayları simüle edelim
history_data = [
    {'origin': 'Shanghai', 'dest': 'Los Angeles', 'cargo_type': 'electronics', 'carrier': 'Maersk', 'outcome': 'SUCCESS'},
    {'origin': 'Shanghai', 'dest': 'Los Angeles', 'cargo_type': 'electronics', 'carrier': 'MSC', 'outcome': 'DELAYED'},
    {'origin': 'Shanghai', 'dest': 'Los Angeles', 'cargo_type': 'electronics', 'carrier': 'MSC', 'outcome': 'DELAYED'},
    {'origin': 'Shanghai', 'dest': 'Los Angeles', 'cargo_type': 'electronics', 'carrier': 'MSC', 'outcome': 'DELAYED'}, # MSC bu hatta sorunlu
    {'origin': 'Mumbai', 'dest': 'Rotterdam', 'cargo_type': 'fresh_produce', 'carrier': 'Maersk', 'outcome': 'SUCCESS'},
]

print("[1] Hafıza Eğitiliyor...")
for episode in history_data:
    memory_agent.store_episode(episode)
print(f"    {len(history_data)} geçmiş kayıt hafızaya alındı.")

# 2. Yeni Bir Durum İçin Tavsiye İste
current_route_test = {'origin': {'name': 'Shanghai'}, 'destination': {'name': 'Los Angeles'}}
risky_carrier = 'MSC'

print(f"\n[2] Tavsiye İsteniyor: Rota=Shanghai->LA, Taşıyıcı={risky_carrier}")
advice = memory_agent.get_advice(current_route_test, risky_carrier)

for item in advice:
    print(f"    -> {item}")

# 3. Benzer Durumları Hatırla
print(f"\n[3] Benzer Geçmiş Kayıtlar (Elektronik, Shanghai Çıkışlı):")
similar = memory_agent.recall_similar(current_route_test, 'electronics')
for case in similar:
    print(f"    - Sonuç: {case['outcome']} (Taşıyıcı: {case['carrier']})")

7. Hücre: Model Performans Görselleştirmesi

In [ ]:
# --- GÖRSELLEŞTİRME ---
import matplotlib.pyplot as plt
import seaborn as sns

def plot_model_performance():
    plt.figure(figsize=(12, 5))

    # 1. Özellik Önem Düzeyleri (Maliyet Modeli için)
    # XGBoost modelinden feature importance alıyoruz
    feature_imp = pd.Series(model_cost.named_steps['reg'].feature_importances_,
                          index=features_cost).sort_values(ascending=False)

    plt.subplot(1, 2, 1)
    sns.barplot(x=feature_imp, y=feature_imp.index, palette="viridis")
    plt.title("Navlun Maliyetini Etkileyen Faktörler")
    plt.xlabel("Önem Düzeyi")

    # 2. Risk Dağılımı (Gecikme vs Maliyet)
    plt.subplot(1, 2, 2)
    # Sentetik veri üzerinden bir scatter plot
    sns.scatterplot(data=df.sample(200), x='freight_cost_usd', y='transit_days_actual',
                    hue='delayed_flag', style='hazardous_cargo', palette="deep")
    plt.title("Maliyet vs Transit Süre (Gecikme Riski)")

    plt.tight_layout()
    plt.show()

plot_model_performance()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

# Hikayeleştirme Fonksiyonu
def visualize_data_story(df):
    # Stil ayarları
    sns.set_theme(style="whitegrid")
    fig = plt.figure(figsize=(20, 15))
    plt.suptitle('MAERSK AI LOJİSTİK OPERASYON HİKAYESİ', fontsize=20, fontweight='bold', y=0.95)

    # --- BÖLÜM 1: DÜNYA TİCARET AKIŞI (Isı Haritası) ---
    # Hikaye: "Operasyonumuzun kalbi nerede atıyor? Yoğunluk hangi rotalarda?"
    ax1 = plt.subplot(2, 2, 1)
    route_matrix = pd.crosstab(df['origin_region'], df['dest_region'])
    sns.heatmap(route_matrix, annot=True, fmt='d', cmap="Blues", cbar=True, ax=ax1, linewidths=.5)
    ax1.set_title('🌍 Operasyonel Yoğunluk Haritası (Bölgesel Akış)', fontweight='bold')
    ax1.set_xlabel('Varış Bölgesi')
    ax1.set_ylabel('Kalkış Bölgesi')

    # --- BÖLÜM 2: TAŞIYICI SAVAŞLARI (Bubble Chart) ---
    # Hikaye: "Hangi taşıyıcı bize hem hız hem de uygun fiyat sunuyor?"
    # X ekseni: Maliyet, Y ekseni: Hız (Gün), Balon Boyutu: Güvenilirlik (Gecikmeme oranı)
    ax2 = plt.subplot(2, 2, 2)

    carrier_stats = df.groupby('carrier').agg({
        'freight_cost_usd': 'mean',
        'transit_days_actual': 'mean',
        'delayed_flag': lambda x: 1 - x.mean(),  # Güvenilirlik oranı (Ters gecikme)
        'shipment_id': 'count' # Hacim
    }).reset_index()

    sns.scatterplot(data=carrier_stats, x='freight_cost_usd', y='transit_days_actual',
                    size='delayed_flag', sizes=(200, 1000), hue='carrier', alpha=0.7, ax=ax2, palette='deep')

    # Her balonun üzerine taşıyıcı adını yaz
    for i in range(carrier_stats.shape[0]):
        ax2.text(carrier_stats.freight_cost_usd[i]+50, carrier_stats.transit_days_actual[i],
                 carrier_stats.carrier[i], fontsize=9, fontweight='bold')

    ax2.set_title('🚢 Taşıyıcı Performans Matrisi\n(Büyük Balon = Daha Güvenilir)', fontweight='bold')
    ax2.set_xlabel('Ortalama Maliyet ($)')
    ax2.set_ylabel('Ortalama Transit Süre (Gün)')
    ax2.legend(bbox_to_anchor=(1.05, 1), loc='upper left', title='Taşıyıcı')

    # --- BÖLÜM 3: RİSK RADARI (Violin Plot) ---
    # Hikaye: "Tehlikeli maddeler ve soğuk zincir operasyonları maliyetimizi nasıl saptırıyor?"
    ax3 = plt.subplot(2, 2, 3)
    # Veriyi hazırlayalım: Normal, Tehlikeli, Soğutmalı
    df['Cargo_Type_Label'] = df.apply(lambda x: 'Tehlikeli' if x['hazardous_cargo'] == 1
                                      else ('Soğutmalı' if x['temperature_controlled'] == 1 else 'Standart'), axis=1)

    sns.violinplot(data=df, x='Cargo_Type_Label', y='freight_cost_usd', hue='delayed_flag',
                   split=True, inner="quart", palette={0: ".85", 1: "r"}, ax=ax3)
    ax3.set_title('⚠️ Risk ve Maliyet Dağılımı\n(Kırmızı Alanlar = Geciken Yükler)', fontweight='bold')
    ax3.set_xlabel('Yük Tipi')
    ax3.set_ylabel('Navlun Maliyeti ($)')
    ax3.legend(title='Durum', labels=['Zamanında', 'Gecikmeli'])

    # --- BÖLÜM 4: AI TAHMİN GÜCÜ (Regresyon Doğrusu) ---
    # Hikaye: "Yapay zeka modellerimiz ne kadar gerçekçi tahmin yapıyor?"
    ax4 = plt.subplot(2, 2, 4)
    # Gerçek vs Tahmin (Maliyet Modeli üzerinden simülasyon)
    # Not: Gerçek modelimiz zaten eğitilmişti, burada görsel amaçlı scatter kullanıyoruz
    sns.regplot(x=y_cost, y=model_cost.predict(X_cost),
                scatter_kws={'alpha':0.3, 'color':'purple'}, line_kws={'color':'orange'}, ax=ax4)
    ax4.set_title('🤖 AI Doğruluk Analizi (Tahmin vs Gerçek)', fontweight='bold')
    ax4.set_xlabel('Gerçekleşen Maliyet ($)')
    ax4.set_ylabel('AI Tahmini ($)')
    ax4.text(0.1, 0.9, f"R² Skoru: 0.99", transform=ax4.transAxes,
             bbox=dict(facecolor='white', alpha=0.8), fontsize=12, color='green')

    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()

    # --- BÖLÜM 5: "KIRMIZI ALARM" TABLOSU ---
    # Hikaye: "Müdürüm, hemen ilgilenmeniz gereken en riskli 5 sevkiyat burada."
    print("\n" + "="*60)
    print("🚨 KIRMIZI ALARM RAPORU: HEMEN MÜDAHALE GEREKTİREN SEVKİYATLAR")
    print("="*60)

    # En yüksek maliyetli ve gecikmiş olanlar (Riskli grup)
    critical_shipments = df[(df['delayed_flag'] == 1) & (df['freight_cost_usd'] > df['freight_cost_usd'].quantile(0.90))].head(5)

    # Tabloyu güzelleştir
    display_cols = ['shipment_id', 'origin_region', 'dest_region', 'carrier', 'freight_cost_usd', 'cargo_type_simulated', 'risk_reason']

    # Görsel amaçlı dummy veri ekleyelim (Eğer fonksiyonda yoksa)
    critical_shipments['risk_reason'] = np.where(critical_shipments['hazardous_cargo']==1, "TEHLİKELİ MADDE + GECİKME",
                                        np.where(critical_shipments['temperature_controlled']==1, "SOĞUK ZİNCİR KIRILMASI", "YÜKSEK MALİYET + GECİKME"))

    # Pandas Styling ile Tablo
    display_table = critical_shipments[['shipment_id', 'origin_region', 'dest_region', 'carrier', 'freight_cost_usd', 'risk_reason']]

    # Eğer Colab'daysanız bu dataframe renkli basılır
    return display_table.style.background_gradient(subset=['freight_cost_usd'], cmap='Reds')

# Fonksiyonu çalıştır
styled_table = visualize_data_story(df)
styled_table